In [14]:
import time, threading

# 新线程执行的代码:
def loop():
    print("thread %s is running..." % threading.current_thread().name)
    n = 0
    while n < 5:
        n += 1
        print("thread %s >>> %s" % (threading.current_thread().name, n))
        time.sleep(1)
    print("thread %s ended." % threading.current_thread().name)
    
print("thread %s is running..." % threading.current_thread().name)
t = threading.Thread(target= loop, name = "LoopThread")
t.start()
t.join()
print("thread %s ended." % threading.current_thread().name)

thread MainThread is running...
thread LoopThread is running...
thread LoopThread >>> 1
thread LoopThread >>> 2
thread LoopThread >>> 3
thread LoopThread >>> 4
thread LoopThread >>> 5
thread LoopThread ended.
thread MainThread ended.


## 进程数据独有，线程数据共享

In [9]:
import time, threading

# 假定这是你的银行存款:
balance = 0
lock = threading.Lock()

def change_it(n):
    # 先存后取，结果应该为0:
    global balance
    balance = balance + n
    balance = balance - n

def run_thread(n):
    for i in range(100000):
        # 先要获取锁:
        lock.acquire()
        try:
            # 放心地改吧:
            change_it(n)
        finally:
            # 改完了一定要释放锁:
            lock.release()

t1 = threading.Thread(target=run_thread, args=(5,))
t2 = threading.Thread(target=run_thread, args=(8,))
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

0


## ThreadLocal

In [3]:
import threading

'''创建全局ThreadLocal对象'''
local_school = threading.local()

def process_student():
    '''获取当前线程关联的student'''
    std = local_school.student
    print("Hello, %s (in %s)" % (std, threading.current_thread().name))
    
def process_thread(name):
    '''绑定ThreadLocal的的student'''
    local_school.student = name
    process_student()
    
t1 = threading.Thread(target = process_thread, args = ("Alice",), name = "Thread-A")
t2 = threading.Thread(target = process_thread, args = ("Bob",), name = "Thread-B")
t1.start()
t2.start()
t1.join()
t2.join()

Hello, Alice (in Thread-A)
Hello, Bob (in Thread-B)


#### 全局变量`local_school`就是一个`ThreadLocal`对象，每个`Thread`对它都可以读写`student`属性，但互不影响。你可以把`local_school`看成全局变量，但每个属性如`local_school.student`都是线程的局部变量，可以任意读写而互不干扰，也不用管理锁的问题，`ThreadLocal`内部会处理。

#### 可以理解为全局变量`local_school`是一个`dict`，不但可以用`local_school.student`，还可以绑定其他变量，如`local_school.teacher`等等。

#### `ThreadLocal`最常用的地方就是为每个线程绑定一个数据库连接，`HTTP`请求，用户身份信息等，这样一个线程的所有调用到的处理函数都可以非常方便地访问这些资源。

In [1]:
import _thread, time
# 定义线程函数
def print_time(threadName, delay):
    count = 0
    while count < 5:
        time.sleep(delay)
        count += 1
        # 返回当前时间的时间戳（1970纪元后经过的浮点秒数）, 并格式化输出
        print("{}: {}".format(threadName, time.ctime(time.time()) ))
try:
    _thread.start_new_thread( print_time, ("Thread-1", 2))
    _thread.start_new_thread( print_time, ("Thread-2", 4))
except:
    print("Error")

while 1:
    # 让线程有足够的时间完成
    pass

Thread-1: Sun Mar 17 14:40:29 2019
Thread-2: Sun Mar 17 14:40:31 2019
Thread-1: Sun Mar 17 14:40:31 2019
Thread-1: Sun Mar 17 14:40:33 2019
Thread-2: Sun Mar 17 14:40:35 2019
Thread-1: Sun Mar 17 14:40:35 2019
Thread-1: Sun Mar 17 14:40:37 2019
Thread-2: Sun Mar 17 14:40:39 2019
Thread-2: Sun Mar 17 14:40:43 2019
Thread-2: Sun Mar 17 14:40:47 2019


KeyboardInterrupt: 